In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime

In [ ]:
# Upload the required files
from google.colab import files
files.upload()

In [ ]:
# Read and display the 'emails sent' file
df_emails = pd.read_excel('Emails sent.xlsx')

# Rename columns to match the other dataframe and add a full name column
df_emails.rename(columns={'name_first':'first_name', 'name_last':'last_name'}, inplace=True)
df_emails["full_name"] = df_emails["first_name"] + " " + df_emails["last_name"]
df_emails

,last_name,first_name,name_middle,mail_sent_date,office,full_name
0,ADELGLASS,JEFFREY,M.,2009-05-22 00:00:00,Center for Drug Evaluation and Research,JEFFREY ADELGLASS
1,ADKINSON,N.,FRANKLIN,2010-04-17,Center for Biologics Evaluation and Research,N. ADKINSON
2,ALLEN,MARK,S.,2012-01-26,Center for Devices and Radiological Health,MARK ALLEN
3,AMSTERDAM,DANIEL,NaN,2014.11.15,Center for Biologics Evaluation and Research,DANIEL AMSTERDAM
4,AMSTUTZ,HARLAN,C.,2014.07.17,Center for Devices and Radiological Health,HARLAN AMSTUTZ
...,...,...,...,...,...,...
267,YADAV,SANJAY,S.,2015-04-09,Center for Devices and Radiological Health,SANJAY YADAV
268,YANNEY,JAMES,NaN,2013-07-27,Center for Devices and Radiological Health,JAMES YANNEY
269,ZHANG,JINGWU,NaN,2009-03-23 00:00:00,Center for Biologics Evaluation and Research,JINGWU ZHANG
270,ZIRAN,BRUCE,H.,2018-08-25,Center for Devices and Radiological Health,BRUCE ZIRAN


In [ ]:
# Read and display the 'fines paid by doctors' file
df_fines = pd.read_csv('Fines paid  by doctors.csv')

# Add a full name column similar to the other file
df_fines["full_name"] = df_fines["first_name"] + " " + df_fines["last_name"]
df_fines

,org_indiv,first_name,last_name,city,state,category,cash_fine,online_fine,full_name
0,3-D MEDICAL SERVICES LLC,STEVEN,DEITELZWEIG,NEW ORLEANS,LA,Professional Advising,"$2,625.00",$-,STEVEN DEITELZWEIG
1,"AA DOCTORS, INC.",AAKASH,AHUJA,PASO ROBLES,CA,Expert-Led Forums,"$1,000.00",$-,AAKASH AHUJA
2,"ABBO, LILIAN MARGARITA",LILIAN,ABBO,MIAMI,FL,Business Related Travel,$-,$448.00,LILIAN ABBO
3,"ABBO, LILIAN MARGARITA",LILIAN,ABBO,MIAMI,FL,Meals,$-,$119.00,LILIAN ABBO
4,"ABBO, LILIAN MARGARITA",LILIAN,ABBO,MIAMI,FL,Professional Advising,"$1,800.00",$-,LILIAN ABBO
...,...,...,...,...,...,...,...,...,...
10082,"ZUWALLACK, RICHARD LEO",RICHARD,ZUWALLACK,HARTFORD,CT,Expert-Led Forums,"$1,750.00",$-,RICHARD ZUWALLACK
10083,"ZUWALLACK, RICHARD LEO",RICHARD,ZUWALLACK,HARTFORD,CT,Meals,$-,$65.00,RICHARD ZUWALLACK
10084,"ZUZARTE, EDWARD",EDWARD,ZUZARTE,ROSEDALE,MD,Business Related Travel,$-,$61.00,EDWARD ZUZARTE
10085,"ZUZARTE, EDWARD",EDWARD,ZUZARTE,ROSEDALE,MD,Expert-Led Forums,"$2,000.00",$-,EDWARD ZUZARTE


# Standardise the mail_sent_date column values to YYYY-MM-DD standard date format

In [ ]:
# Create a copy file to be used for further operations
df_emails2 = df_emails.copy()

In [ ]:
df_emails2['mail_sent_date'] = pd.to_datetime(df_emails2['mail_sent_date']).dt.date
df_emails2

,last_name,first_name,name_middle,mail_sent_date,office,full_name
0,ADELGLASS,JEFFREY,M.,2009-05-22,Center for Drug Evaluation and Research,JEFFREY ADELGLASS
1,ADKINSON,N.,FRANKLIN,2010-04-17,Center for Biologics Evaluation and Research,N. ADKINSON
2,ALLEN,MARK,S.,2012-01-26,Center for Devices and Radiological Health,MARK ALLEN
3,AMSTERDAM,DANIEL,NaN,2014-11-15,Center for Biologics Evaluation and Research,DANIEL AMSTERDAM
4,AMSTUTZ,HARLAN,C.,2014-07-17,Center for Devices and Radiological Health,HARLAN AMSTUTZ
...,...,...,...,...,...,...
267,YADAV,SANJAY,S.,2015-04-09,Center for Devices and Radiological Health,SANJAY YADAV
268,YANNEY,JAMES,NaN,2013-07-27,Center for Devices and Radiological Health,JAMES YANNEY
269,ZHANG,JINGWU,NaN,2009-03-23,Center for Biologics Evaluation and Research,JINGWU ZHANG
270,ZIRAN,BRUCE,H.,2018-08-25,Center for Devices and Radiological Health,BRUCE ZIRAN


# The year in which maximum number of emails were sent to doctors

In [ ]:
pd.DataFrame(df_emails2.groupby(df_emails2['mail_sent_date'].apply(lambda x: x.year)).size().sort_values(ascending=False), columns=['emails'])

,emails
mail_sent_date,
2014,32
2015,31
2019,25
2016,25
2010,25
2018,23
2013,21
2011,20
2012,16


# Total fine paid per Office

In [ ]:
# Create a copy file to be used for further operations
df_fines2 = df_fines.copy()

**Data Cleaning**

In [ ]:
# Clean the first_name and last_name columns by only considering alphabets
df_fines2['first_name'] = np.where(df_fines2['first_name'].str.count(r'[^a-zA-Z]') > 0, "", df_fines2['first_name'])
df_fines2['last_name'] = np.where(df_fines2['last_name'].str.count(r'[^a-zA-Z]') > 0, "", df_fines2['last_name'])

# Clean the cash_fine and online_fine columns by removing special symbols other than period and then convert the values to float
df_fines2[' cash_fine '] = df_fines2[' cash_fine '].str.replace(r'[^0-9.]', '')
df_fines2[' cash_fine '] = pd.to_numeric(df_fines2[' cash_fine '].astype(str).str[:-3])

df_fines2[' online_fine '] = df_fines2[' online_fine '].str.replace(r'[^0-9.]', '')
df_fines2[' online_fine '] = pd.to_numeric(df_fines2[' online_fine '].astype(str).str[:-3])
df_fines2

,org_indiv,first_name,last_name,city,state,category,cash_fine,online_fine,full_name
0,3-D MEDICAL SERVICES LLC,STEVEN,DEITELZWEIG,NEW ORLEANS,LA,Professional Advising,2625.0,NaN,STEVEN DEITELZWEIG
1,"AA DOCTORS, INC.",AAKASH,AHUJA,PASO ROBLES,CA,Expert-Led Forums,1000.0,NaN,AAKASH AHUJA
2,"ABBO, LILIAN MARGARITA",LILIAN,ABBO,MIAMI,FL,Business Related Travel,NaN,448.0,LILIAN ABBO
3,"ABBO, LILIAN MARGARITA",LILIAN,ABBO,MIAMI,FL,Meals,NaN,119.0,LILIAN ABBO
4,"ABBO, LILIAN MARGARITA",LILIAN,ABBO,MIAMI,FL,Professional Advising,1800.0,NaN,LILIAN ABBO
...,...,...,...,...,...,...,...,...,...
10082,"ZUWALLACK, RICHARD LEO",RICHARD,ZUWALLACK,HARTFORD,CT,Expert-Led Forums,1750.0,NaN,RICHARD ZUWALLACK
10083,"ZUWALLACK, RICHARD LEO",RICHARD,ZUWALLACK,HARTFORD,CT,Meals,NaN,65.0,RICHARD ZUWALLACK
10084,"ZUZARTE, EDWARD",EDWARD,ZUZARTE,ROSEDALE,MD,Business Related Travel,NaN,61.0,EDWARD ZUZARTE
10085,"ZUZARTE, EDWARD",EDWARD,ZUZARTE,ROSEDALE,MD,Expert-Led Forums,2000.0,NaN,EDWARD ZUZARTE


In [ ]:
# Create a copy here to be used for getting the 'top 5 average fine paid per state and category' at the end of this notebook
df_fines_last = df_fines2.copy()

In [ ]:
# Perform groupby operation for the following columns and add a total fine column
df_fines2 = df_fines2.groupby(['org_indiv','first_name','last_name','full_name','state'], as_index=False).sum()
df_fines2['total_fine'] = df_fines2.apply(lambda x: x[' cash_fine '] + x[' online_fine '], axis=1)
df_fines2

,org_indiv,first_name,last_name,full_name,state,cash_fine,online_fine,total_fine
0,3-D MEDICAL SERVICES LLC,STEVEN,DEITELZWEIG,STEVEN DEITELZWEIG,LA,2625.0,0.0,2625.0
1,"AA DOCTORS, INC.",AAKASH,AHUJA,AAKASH AHUJA,CA,1000.0,0.0,1000.0
2,"ABBO, LILIAN MARGARITA",LILIAN,ABBO,LILIAN ABBO,FL,1800.0,567.0,2367.0
3,ABDULLAH RAFFEE MD PC,ABDULLAH,RAFFEE,ABDULLAH RAFFEE,MI,750.0,0.0,750.0
4,"ABEBE, SHEILA Y",SHEILA,ABEBE,SHEILA ABEBE,IN,825.0,47.0,872.0
...,...,...,...,...,...,...,...,...
4712,"ZUB, DAVID WILLIAM",DAVID,ZUB,DAVID ZUB,NC,47.0,1236.0,1283.0
4713,"ZUBAL, BETH ANN",BETH,ZUBAL,BETH ZUBAL,MO,2100.0,562.0,2662.0
4714,"ZUSMAN, RANDALL MARK",RANDALL,ZUSMAN,RANDALL ZUSMAN,MA,0.0,500.0,500.0
4715,"ZUWALLACK, RICHARD LEO",RICHARD,ZUWALLACK,RICHARD ZUWALLACK,CT,1750.0,65.0,1815.0


In [ ]:
# Create multiple copies to be used for other operations
df_fines3 = df_fines2.copy()
df_fines4 = df_fines2.copy()
df_fines5 = df_fines2.copy()

df_emails3 = df_emails2.copy()

In [ ]:
# Drop columns not needed for analysis
df_fines3.drop(['org_indiv','first_name','last_name',' cash_fine ',' online_fine '], axis=1, inplace=True)
df_emails3.drop(['last_name','first_name','name_middle','mail_sent_date'], axis=1, inplace=True)

In [ ]:
# Marge the 'df_fines3' & 'df_emails3' dataframes
df_merge = pd.merge(df_fines3, df_emails3, on='full_name')
df_merge

,full_name,state,total_fine,office
0,ALAN RAPOPORT,CA,7545.0,Center for Drug Evaluation and Research
1,RONALD BUKOWSKI,OH,72425.0,Center for Drug Evaluation and Research
2,RONALD BUKOWSKI,OH,9641.0,Center for Drug Evaluation and Research
3,THOMAS GAZDA,AZ,2363.0,Center for Drug Evaluation and Research
4,THOMAS GAZDA,AZ,10001.0,Center for Drug Evaluation and Research
5,JEFFREY LEVENSON,FL,1792.0,Center for Drug Evaluation and Research
6,ROBERT RITCH,NY,15364.0,Center for Devices and Radiological Health
7,ROBERT RITCH,NY,15364.0,Center for Devices and Radiological Health
8,ADAM SMITH,NY,581.0,Center for Devices and Radiological Health
9,DOUGLAS WARD,DC,2896.0,Center for Biologics Evaluation and Research


In [ ]:
# Print the total fine paid per office
df_merge.groupby(['office'], as_index=False).sum()

,office,total_fine
0,Center for Biologics Evaluation and Research,2896.0
1,Center for Devices and Radiological Health,31309.0
2,Center for Drug Evaluation and Research,103767.0


# Get the highest total fine and lowest total fine paid per state

In [ ]:
df_fines3.drop(['full_name'], axis=1, inplace=True)

**Highest total fine paid per state**

In [ ]:
df_fines3.groupby(['state'], as_index=False).max()

,state,total_fine
0,AK,1750.0
1,AL,66096.0
2,AR,10504.0
3,AZ,103104.0
4,CA,252745.0
5,CO,53051.0
6,CT,90000.0
7,DC,101828.0
8,DE,22795.0
9,FL,1185466.0


**Lowest total fine paid per state**

In [ ]:
df_fines3.groupby(['state'], as_index=False).min()

,state,total_fine
0,AK,1750.0
1,AL,588.0
2,AR,586.0
3,AZ,625.0
4,CA,502.0
5,CO,522.0
6,CT,551.0
7,DC,750.0
8,DE,562.0
9,FL,77.0


# The least 2 states that had to pay minimum 'online' fine

In [ ]:
df_fines4.drop(['org_indiv','first_name','last_name','full_name',' cash_fine ','total_fine'], axis=1, inplace=True)

In [ ]:
df_fines4 = df_fines4.groupby(['state'], as_index=False).sum()
df_fines4.sort_values(by=' online_fine ', ascending=True).head(2)

,state,online_fine
0,AK,0.0
28,ND,321.0


# The number of doctors that contributed to the state having highest 'cash' fine

In [ ]:
df_fines5.drop(['org_indiv','first_name','last_name',' online_fine ','total_fine'], axis=1, inplace=True)

In [ ]:
# Create a copy of the above file to later count the number of doctors for a particular state
df_fines5_doc = df_fines5.copy()

In [ ]:
# Find the state having the highest cash fine
df_fines5 = df_fines5.groupby(['state'], as_index=False).sum()
df_fines5.sort_values(by=' cash_fine ', ascending=False).head(1)

,state,cash_fine
4,CA,3205893.0


In [ ]:
# Count the numbers of doctors in that state having the highest cash fine by using the above copy file
df_fines5_doc['full_name'].loc[df_fines5_doc['state'] == 'CA'].count()

554

# Get the top 5 average fine paid per state and category

In [ ]:
# Treat the NaN values in both the fines columns and create a column for total fines
df_fines_last[' cash_fine '] = df_fines_last[' cash_fine '].fillna(0)
df_fines_last[' online_fine '] = df_fines_last[' online_fine '].fillna(0)
df_fines_last['total_fine'] = df_fines_last.apply(lambda x: x[' cash_fine '] + x[' online_fine '], axis=1)

**Top 5 average fine paid per state**

In [ ]:
pd.DataFrame(df_fines_last.groupby('state')['total_fine'].nlargest(5))

total_fine
state                 
AK    2364      1750.0
AL    9256    232991.0
      9257     66096.0
      9255     55937.0
      8576     46494.0
...                ...
WY    9336      7750.0
      4000      7425.0
      5910      5200.0
      7034      5125.0
      1742      2625.0

[256 rows x 1 columns]

**Top 5 average fine paid per category**

In [ ]:
pd.DataFrame(df_fines_last.groupby('category')['total_fine'].nlargest(5))

total_fine
category                                                           
Business Related Travel                            5193     27681.0
                                                   6975     21100.0
                                                   6278     18573.0
                                                   818      18063.0
                                                   4564     14183.0
Educational Items                                  2661       500.0
                                                   4348       500.0
                                                   7195       500.0
                                                   1721        99.0
                                                   2378        99.0
Expert-Led Forums                                  7772    146500.0
                                                   3798     79000.0
                                                   5943     78000.0
                                                   9600     75400.0
                                                   6397     70500.0
Infosys Sponsored Research initiated before Jul... 2331    857698.0
                                                   9299    672263.0
                                                   1966    617548.0
                                                   9314    479314.0
                                                   7340    323122.0
Infosys Sponsored Research initiated on or afte... 6160   1185466.0
                                                   7832    252745.0
                                                   9113    177786.0
                                                   2332    156000.0
                                                   4223    152466.0
Investigator-Initiated Research                    9311    162900.0
                                                   9296    150778.0
                                                   9312     94800.0
                                                   9201     92000.0
                                                   6455     90000.0
Meals                                              1141      4189.0
                                                   7773      4001.0
                                                   7471      3796.0
                                                   9601      3679.0
                                                   3663      3071.0
Professional Advising                              461     105000.0
                                                   7955    100000.0
                                                   5505     87610.0
                                                   9751     85621.0
                                                   8996     75609.0